# Import necessary modules

In [ ]:
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets.cifar10 import load_data
from tensorflow.keras.applications import mobilenet
import numpy as np
import tensorflow as tf

# Load Dataset

In [ ]:
(trainX, trainY), (testX, testY) = load_data()

trainX = tf.image.resize(trainX, (224, 224))
testX = tf.image.resize(testX, (224, 224))

trainY = to_categorical(trainY)
testY = to_categorical(testY)

print(f"Train: X={trainX.shape}, Y={trainY.shape}")
print(f"Test: X={testX.shape}, Y={testY.shape}")

170498071/170498071 [==============================] - 3s 0us/step
Train: X=(50000, 224, 224, 3), Y=(50000, 10)
Test: X=(10000, 224, 224, 3), Y=(10000, 10)


# Build CNN

In [ ]:
base_model = mobilenet.MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
x = base_model.output
x = Flatten()(x)
x = Dense(16, activation = 'relu')(x)
x = Dense(32, activation = 'relu')(x)
x = Dense(64, activation = 'relu')(x)
outputs = Dense(10, activation = 'softmax')(x)
model = Model(inputs = base_model.input, outputs = outputs)
model.summary()

17225924/17225924 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                              

# Train classifier without validation dataset.

In [ ]:
model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(trainX, trainY, epochs = 10, batch_size = 256)

print(f"Loss: {model.evaluate(testX, testY)[0]}")
print(f"Accuracy: {model.evaluate(testX, testY)[1]}")

Epoch 1/10
  7/196 [>.............................] - ETA: 33:04 - loss: 2.0665 - accuracy: 0.1786

KeyboardInterrupt: 

# Test the performance of the model

In [ ]:
model.evaluate(testX, testY)
predictY = model.predict(testX)

print('OriginalY  PredictedY')
for i in range(10):
    print(f'{np.argmax(testY[i])}          {np.argmax(predictY[i])}')

# Training CNN with Validation

In [ ]:
model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(trainX, trainY, epochs = 10, batch_size = 128, validation_split = 0.2)


print(f"Loss: {model.evaluate(testX, testY)[0]}")
print(f"Accuracy: {model.evaluate(testX, testY)[1]}")